In [8]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

In [9]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [12]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32


In [13]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [14]:
IMG_HEIGHT, IMG_WIDTH = 299, 299

In [15]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-25 21:16:59.344346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 21:16:59.349453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 21:16:59.353176: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [16]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [17]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.inception_resnet_v2.preprocess_input

In [18]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [19]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [20]:
import tensorflow.keras.applications as apps
base_model = apps.InceptionResNetV2(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [21]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: conv2d, False
Layer: batch_normalization, False
Layer: activation, False
Layer: conv2d_1, False
Layer: batch_normalization_1, False
Layer: activation_1, False
Layer: conv2d_2, False
Layer: batch_normalization_2, False
Layer: activation_2, False
Layer: max_pooling2d, False
Layer: conv2d_3, False
Layer: batch_normalization_3, False
Layer: activation_3, False
Layer: conv2d_4, False
Layer: batch_normalization_4, False
Layer: activation_4, False
Layer: max_pooling2d_1, False
Layer: conv2d_8, False
Layer: batch_normalization_8, False
Layer: activation_8, False
Layer: conv2d_6, False
Layer: conv2d_9, False
Layer: batch_normalization_6, False
Layer: batch_normalization_9, False
Layer: activation_6, False
Layer: activation_9, False
Layer: average_pooling2d, False
Layer: conv2d_5, False
Layer: conv2d_7, False
Layer: conv2d_10, False
Layer: conv2d_11, False
Layer: batch_normalization_5, False
Layer: batch_normalization_7, False
Layer: batch_normalization_10, False
Lay

In [22]:
len(base_model.layers)

780

In [23]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [29]:
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
x = base_model(inputs, training = False)
x = layers.GlobalAvgPool2D()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'InceptionResnetV2')

model_1.summary()

Model: "InceptionResnetV2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 inception_resnet_v2 (Funct  (None, 8, 8, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 512)               786944    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 512)         

In [30]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [31]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=2) 

In [32]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [33]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 10, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/10


2024-07-25 21:17:49.591493: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-25 21:17:49.657069: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 1/74 [..............................] - ETA: 8:12 - loss: 1.7512 - accuracy: 0.2500

2024-07-25 21:17:51.371816: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x63e0aa240a40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-25 21:17:51.371849: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-25 21:17:51.379531: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-25 21:17:51.443673: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 1.2511 - accuracy: 0.4227
Epoch 1: val_loss improved from inf to 1.29353, saving model to Trained_Models/InceptionResnetV2
74/74 [==============================] - 41s 464ms/step - loss: 1.2511 - accuracy: 0.4227 - val_loss: 1.2935 - val_accuracy: 0.0633 - lr: 0.0010
Epoch 2/10
74/74 [==============================] - ETA: 0s - loss: 0.8764 - accuracy: 0.5947
Epoch 2: val_loss improved from 1.29353 to 1.01725, saving model to Trained_Models/InceptionResnetV2
74/74 [==============================] - 31s 420ms/step - loss: 0.8764 - accuracy: 0.5947 - val_loss: 1.0173 - val_accuracy: 0.5806 - lr: 0.0010
Epoch 3/10
74/74 [==============================] - ETA: 0s - loss: 0.7775 - accuracy: 0.6789
Epoch 3: val_loss improved from 1.01725 to 0.64071, saving model to Trained_Models/InceptionResnetV2
74/74 [==============================] - 31s 412ms/step - loss: 0.7775 - accuracy: 0.6789 - val_loss: 0.6407 - val_accuracy: 0.8981 - lr: 0.

In [34]:
base_model.trainable = True

In [35]:
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [36]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [37]:
start_epoch = 10
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0005), metrics = ['accuracy'])
model_1.summary()

Model: "InceptionResnetV2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 inception_resnet_v2 (Funct  (None, 8, 8, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 512)               786944    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 512)         

In [38]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 11/20
74/74 [==============================] - ETA: 0s - loss: 2.4855 - accuracy: 0.5993
Epoch 11: val_loss did not improve from 0.48134
74/74 [==============================] - 18s 135ms/step - loss: 2.4855 - accuracy: 0.5993 - val_loss: 0.5957 - val_accuracy: 0.8160 - lr: 5.0000e-04
Epoch 12/20
74/74 [==============================] - ETA: 0s - loss: 0.4873 - accuracy: 0.8204
Epoch 12: val_loss improved from 0.48134 to 0.37528, saving model to Trained_Models/InceptionResnetV2
74/74 [==============================] - 34s 466ms/step - loss: 0.4873 - accuracy: 0.8204 - val_loss: 0.3753 - val_accuracy: 0.9486 - lr: 5.0000e-04
Epoch 13/20
74/74 [==============================] - ETA: 0s - loss: 0.3451 - accuracy: 0.8691
Epoch 13: val_loss did not improve from 0.37528
74/74 [==============================] - 9s 113ms/step - loss: 0.3451 - accuracy: 0.8691 - val_loss: 0.5124 - val_accuracy: 0.8249 - lr: 5.0000e-04
Epoch 14/20
74/74 [==============================] - ETA: 0s - loss: 0.

In [42]:
start_epoch = 20
for layer in base_model.layers[-150:]:
    layer.trainable = True

In [43]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "InceptionResnetV2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 inception_resnet_v2 (Funct  (None, 8, 8, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 512)               786944    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 512)         

In [44]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/30
74/74 [==============================] - ETA: 0s - loss: 0.6330 - accuracy: 0.8014
Epoch 21: val_loss did not improve from 0.20662
74/74 [==============================] - 22s 190ms/step - loss: 0.6330 - accuracy: 0.8014 - val_loss: 0.3054 - val_accuracy: 0.9327 - lr: 3.0000e-04
Epoch 22/30
74/74 [==============================] - ETA: 0s - loss: 0.2950 - accuracy: 0.8971
Epoch 22: val_loss improved from 0.20662 to 0.19147, saving model to Trained_Models/InceptionResnetV2
74/74 [==============================] - 36s 492ms/step - loss: 0.2950 - accuracy: 0.8971 - val_loss: 0.1915 - val_accuracy: 0.9387 - lr: 3.0000e-04
Epoch 23/30
74/74 [==============================] - ETA: 0s - loss: 0.2024 - accuracy: 0.9314
Epoch 23: val_loss did not improve from 0.19147
74/74 [==============================] - 10s 124ms/step - loss: 0.2024 - accuracy: 0.9314 - val_loss: 0.2295 - val_accuracy: 0.9228 - lr: 3.0000e-04
Epoch 24/30
74/74 [==============================] - ETA: 0s - loss: 0

In [45]:
start_epoch = 30
for layer in base_model.layers[-400:]:
    layer.trainable = True

In [46]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0001), metrics = ['accuracy'])
model_1.summary()

Model: "InceptionResnetV2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 inception_resnet_v2 (Funct  (None, 8, 8, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 512)               786944    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 512)         

In [47]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+8, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 31/38
74/74 [==============================] - ETA: 0s - loss: 0.4077 - accuracy: 0.9140
Epoch 31: val_loss did not improve from 0.13797
74/74 [==============================] - 32s 195ms/step - loss: 0.4077 - accuracy: 0.9140 - val_loss: 0.3088 - val_accuracy: 0.9268 - lr: 1.0000e-04
Epoch 32/38
74/74 [==============================] - ETA: 0s - loss: 0.1364 - accuracy: 0.9564
Epoch 32: val_loss did not improve from 0.13797
74/74 [==============================] - 13s 170ms/step - loss: 0.1364 - accuracy: 0.9564 - val_loss: 0.1730 - val_accuracy: 0.9693 - lr: 1.0000e-04
Epoch 33/38
74/74 [==============================] - ETA: 0s - loss: 0.1315 - accuracy: 0.9593
Epoch 33: val_loss did not improve from 0.13797
74/74 [==============================] - 13s 171ms/step - loss: 0.1315 - accuracy: 0.9593 - val_loss: 0.1921 - val_accuracy: 0.9594 - lr: 1.0000e-04
Epoch 34/38
74/74 [==============================] - ETA: 0s - loss: 0.0523 - accuracy: 0.9848
Epoch 34: val_loss did not im

In [48]:
start_epoch = 38
for layer in base_model.layers[-770:]:
    layer.trainable = True

In [49]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0000015), metrics = ['accuracy'])
model_1.summary()

Model: "InceptionResnetV2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 inception_resnet_v2 (Funct  (None, 8, 8, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 512)               786944    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 512)         

In [50]:
history_250 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+5, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler])

Epoch 39/43
74/74 [==============================] - ETA: 0s - loss: 0.0036 - accuracy: 0.9987
Epoch 39: val_loss did not improve from 0.13797
74/74 [==============================] - 68s 277ms/step - loss: 0.0036 - accuracy: 0.9987 - val_loss: 0.2814 - val_accuracy: 0.9654 - lr: 1.5000e-06
Epoch 40/43
16/74 [=====>........................] - ETA: 12s - loss: 0.0016 - accuracy: 1.0000

KeyboardInterrupt: 

In [51]:
MobileNet_Best = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/InceptionResnetV2")


In [52]:
MobileNet_Best.evaluate(test_datagen)

27/27 [==============================] - 4s 106ms/step - loss: 0.4096 - accuracy: 0.9065


[0.40956565737724304, 0.9065088629722595]